# Прогнозирование числа просмотров объявлений

Автор: Бокиев Наврас <br>
Slack: @bnavras

<img src="https://preview.ibb.co/gKQNPH/photofacefun_com_1524417017.jpg">

#### Импортируем необходимые библиотеки:

In [ ]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.sparse import hstack
from pymorphy2 import MorphAnalyzer
from matplotlib import pyplot as plt
from sklearn.grid_search import GridSearchCV
from xgboost import XGBClassifier, XGBRegressor
from scipy.stats import normaltest, skew, kurtosis
from sklearn.ensemble import RandomForestRegressor
from scikitplot.estimators import plot_learning_curve
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV, Lasso, Ridge

warnings.filterwarnings('ignore')

# Содержание

1. Описание набора данных и признаков
2. Первичный анализ данных
3. Первичный визуальный анализ данных
4. Инсайты и закономерности
5. Выбор метрики
6. Выбор модели
7. Предобработка данных
8. Кросс-валидация и настройка гиперпараметров модели
9. Создание новых признаков и описание этого процесса
10. Построение кривых валидации и обучения
11. Прогноз для отложенной выборке
12. Выводы

# 1. Описание набора данных и признаков

#### Задача: 
Задача проекта - предсказать количество просмотров объявления до конца следующего дня с момента подачи

#### Процесс сбора данных: 
Данные собраны сайтом объявлений Avito. Всего в датасете 423772 объявлений, которые были публикованы с 26 по 28 декабря 2016 года. 
Датасет можно найти на [Boosters](https://boosters.pro/champ_4), либо скачать по [ссылке](https://yadi.sk/d/RY_fdJab3Ue6BK).

#### Ценность задачи:  

Знание количества просмотров может позволить сервису:
- исследовать рынок
- узнать примерную нагрузку на сайт
- оптимизировать сервис
- разместить рекламу на наиболее просматриваемых объявлениях
- выявить объявления, которые наберут небольшое количество просмотров (исследовать причины, предпринять меры) и т.д.


То есть результат поможет сделать сервис лучше!

###  Признаки

- **start_time** - Время подачи объявления (МСК)
- **title** - Заголовок объявления
- **price** - Цена
- **item_id** - Идентификатор объявления
- **owner_type** - Тип владельца объявления
    - *Private* – частный пользователь
    - *Company* – компания
    - *Shop* – владелец магазина на Avito
- **category** - Категория объявления (Транспорт, недвижимость и т.д.)
- **subcategory** - Подкатегория объявления
- **region** - Регион размещения объявления
- **param1**, **param2**, **param3** - Параметры объявления:
    
    На примере подкатегории «Детская одежда и обувь»

    - *param1* = Вид одежды = «Для мальчиков»
    - *param2* = Предмет одежды = “Верхняя одежда»
    - *param3* = Размер = «50-56 cм (0-2 мес)»
    
    
#### Целевая переменная:


`item_views` - количество просмотров объявления

Считаем данные и посмотрим на них:

In [ ]:
data = pd.read_csv('data/task3/train.csv', delimiter=';', index_col=0)  # проверить влияние id, затем удалить
data.head(3)

# 2. Первичный анализ данных

#### Взаимодействия признаков и их влияние на количесто просмотров 

Посмотрим основные статистические характеристики вещественных признаков:

In [ ]:
data[['price', 'item_views']].describe()

Видим, что большинство объявлений не выходит за рамки ста просмотров, при этом максимальное число просмотров равно 4140. Большая разница также между верхней картилью цены и ее максимумом. Возможно, мы имеем дело с выбросами, но вернемся к этому вопросу на этапе фильтрации данных, а пока посмотрим на матрицу корреляций:

In [ ]:
data[['price', 'item_views']].corr()

**Вывод**: переменные слабо связаны.

Среди объявлений есть с ценой, равной нулю. Это, как правило, либо вакансии, либо желание людей избавиться от каких-то вещей, домашних животных.

In [ ]:
data[data.price == 0].head(4)

Также посмотрим характеристики других признаков: 

In [ ]:
data.describe(include=['object']) 

В данных много категориальных признаков. На данном этапе уже можно сделать некоторые выводы о количестве уникальных  значений, моде каждого признака.

#### Распределение целевого признака

In [ ]:
plt.figure(figsize=(10,5))
plt.xlim(0, 400)
plt.hist(data.item_views, bins=1000, alpha=0.5, color='red', label='true');

Проверим на нормальность и скошенность

In [ ]:
normaltest_result = normaltest(data.item_views)
print(f"{normaltest_result.pvalue}") 

In [ ]:
print(f"Скошенность: {skew(data.item_views)}")

Мы не будем обучать линейные и метрические модели, поэтому оставим целевую переменную без преобразований

#### Пропуски

In [ ]:
null_count = data.isnull().sum()
null_count[null_count != 0]

Как видим, пропуски есть только в параметрах. Эти признаки описывают объект объявления. Соответствующие формы могут быть при создании не заполнены, либо товар не описывается какими-либо измерениями. В пользу последнего предположения - возрастание количества пропусков с ростом количества параметров. Заполним пропуски произвольной строкой, непересекающейся с имеющимися признаками.

In [ ]:
data.fillna('Не заполнено');

#### Выбросы

На этапе первичного анализа мы выяснили, что верхняя квантиль цены и количества просмотров не привышают 10000 и 60 соответственно. Но предельные значения этих признаков гораздо больше. На момент исследования на сайте Avito самым дорогим объявлением был земельный участок стоимостью 2 858 076 000 руб. В наших же данных:

In [ ]:
data.price.max()

Посмотрим на несколько предельных значений вещественных признаков

In [ ]:
sorted(data.price, reverse=True)[:5]

In [ ]:
sorted(data.item_views, reverse=True)[:5]

Мы не будем удалять выбросы, потому что модели, которые мы собираемся обучать, устойчивы к ним.

# 3. Первичный визуальный анализ данных

Визуализируем для наглядности матрицу корреляций:

In [ ]:
sns.set(rc={"figure.figsize": (10, 5)})
sns.heatmap(data[['price', 'item_views']].corr(), 
            cmap="YlGnBu",
            annot=True);

Теперь давайте исследуем категориальные признаки и выясним, влияет ли каждый признак на количество просмотров. Мы будем вычислять медиану целевого признака для каждого значения.

Начнем с категорий:

In [ ]:
median_target_for_category = data.groupby(['category']).item_views.median()
plt.xticks(rotation=90)
plt.bar(height=median_target_for_category, x=median_target_for_category.index);

Видно, что разные категории набирают разное количество просмотров. Теперь давайте посмотрим на зависимость цены от целевого признака.

In [ ]:
plt.figure(figsize=(15,5))
plt.xlim(0, 10000000)
plt.scatter(data.price, data.item_views);

Чем дороже объект объявления, тем меньше оно набирает просмотров. 

На следующей диаграмме можно увидеть лидеров по количеству объявлений: 

In [ ]:
plt.figure(figsize=(15,3))
plt.xticks(rotation=90)
sns.countplot(data=data, x='region', 
              order=data.region.value_counts().index);

Но по количеству просмотров лидероми являются совсем другие регионы:

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=90)
median_target_for_region = data.groupby(['region']).item_views.median()
plt.bar(height=median_target_for_region,
        x=median_target_for_region.index);

Разное количество просмотров мы можем обнаружить также для подкатегорий и типов собственника

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=90)
median_target_for_subcategory = data.groupby(['subcategory']).item_views.median()
plt.bar(height=median_target_for_subcategory, x=median_target_for_subcategory.index);

In [ ]:
plt.figure(figsize=(15,5))
median_target_for_owner_type = data.groupby(['owner_type']).item_views.median()
plt.bar(height=median_target_for_owner_type, 
        x=median_target_for_owner_type.index);

Медиана для каждого значения категориальных признаков разная. То есть каждая категория таваров имеет определенную спицифику, что влияет на количество просмотров. 

# 4. Инсайты и закономерности

Обобщим некоторые наблюдения:

- Чем дороже объект объявления, тем меньше просмотров оно набирает 
- Лидеры по количеству объявлений(Москва, Санкт-Петербург) и по количеству просмотров(Чеченская республика, Дагестан) не совпадают.
- Больше просмотров набирают объявления категории Животные, чем Личные вещи
- В целом: среднее число просмотров для каждого значения категориального признака ярко отличаются

К инсайтам и закономерностьям мы еще вернемся на этапе создания новый признаков

# 5. Выбор метрики

В процессе исследования я буду считать две метрики:
- Среднеквадратическое отклонение логорифмов
- Среднюю абсолютную ошибку

Первую будем считать, чтобы ориентироваться на лидербор [соревнования](https://boosters.pro/champ_4).


Вторая метрика более простая и очевидная, выражается в тех же измерениях, что и целевая переменная. Отвечает на главный вопрос, на сколько просмотров мы в среднем ошибаемся?

<img src='https://preview.ibb.co/mVLdWx/Code_Cogs_Eqn.gif'>

In [ ]:
def rmsle(x, y):
    return np.sqrt(np.mean((np.log(x+1)-np.log(y+1))**2))
rmsle_scorer = make_scorer(rmse)

# 6. Выбор модели

Перед нами задача регрессии: cпрогнозировать количество просмотров. Обучать будем следуюшие модели:

- RandomForestRegressor
- XGBoostRegressor

У нас множество категориальных признаков, кроме того данные модели не требуют масштабирования вещественных признаков

# 7. Предобработка данных

In [ ]:
X = data.drop(columns=['item_views'], axis=1)
y = data.item_views

Преобразуем дату в соответствующий тип данных:

In [ ]:
X.start_time = X.start_time.apply(pd.to_datetime)

#### Заменим строки на числа и применим one hot encoder

In [ ]:
def to_ohe(series):
    label_encoder = LabelEncoder()
    one_hot_encoder = OneHotEncoder()
   
    owner_type_train = label_encoder.fit_transform(series)
    return one_hot_encoder.fit_transform(owner_type_train.reshape(-1, 1))

In [ ]:
owner_type_ohe = to_ohe(X.owner_type)
category_ohe = to_ohe(X.category)
region_ohe = to_ohe(X.region)

In [ ]:
param1_ohe = to_ohe(X.param1.astype(str))
param2_ohe = to_ohe(X.param2.astype(str))
param3_ohe = to_ohe(X.param3.astype(str))

#### Bag of words | title

Из заголовка удалим некоторые символы, а слова приведем к нормальной форме. Затем составим мешок из 10000 самых популярых слов

In [ ]:
morph_analyzer = MorphAnalyzer()
def title_preprocess(title):
    title.replace(';', '').replace(',', '')
    return ' '.join([morph_analyzer.normal_forms(word)[0] 
                     for word in title.split()])

In [ ]:
X.title = X.title.apply(title_preprocess)
count_vectorizer = CountVectorizer(max_features=10000)
title = count_vectorizer.fit_transform(X.title)

In [ ]:
price = X.price.values.reshape(-1, 1)

Объединим все признаки:

In [ ]:
X_full = hstack([category_ohe, 
                 owner_type_ohe, 
                 region_ohe, 
                 title,
                 price.reshape(-1,1),
                 param1_ohe,
                 param2_ohe,
                 param3_ohe
                 ])

#### Разобьем данные на обучающую и отложенную части

In [ ]:
X_train, X_holdout, y_train,  y_holdout = train_test_split(X_full.tocsr(), y, test_size=0.3, random_state=17)

# 8. Кросс-валидация

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train);

In [ ]:
xgboost = XGBRegressor(n_estimators=5627, 
                      max_depth=10, 
                      subsample=0.9, 
                      colsample_bytree=0.8, 
                      reg_lambda=0, 
                      reg_alpha=2, 
                      learning_rate=0.05, 
                      seed=250)
xgboost.fit(X_train, y_train);

In [ ]:
predict = rf.predict(X_holdout)
rmse(y_holdout, predict)

In [ ]:
predict = xgboost.predict(X_holdout)
rmse(y_holdout, predict)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
rf_scores = cross_val_score(estimator=rf,
                         X=X_train_full,
                         y=y_train,
                         cv=skf,
                         scoring=rmse_scorer);

In [ ]:
rf_scores.mean()

In [ ]:
xgb_scores = cross_val_score(estimator=xgboost,
                         X=X_train_full,
                         y=y_train,
                         cv=skf,
                         scoring=rmse_scorer);

In [ ]:
xgb_scores.mean()

# 9. Создание новых признаков

Можно предположить, что объявления с длинным заголовком труднее читать и понимать. Скорее всего они не набирать много просмотров. Давайте создадим этот признак и посмотрим на распределение признака:

In [ ]:
title_len = X.title.apply(len)

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(title_len_train, bins=10);

Действительно, есть некоторый оптимум(то есть не очень короткие и не слишком длинные заголовки), который набирает больше просмотров.

Все объявления в датасете опубликованы в интервале с 26 по 28 декабря 2016 года. То есть в публикации все объявления находились разное время. Логично, что объявления, которые были опубликованы раньше, имели преимущество по времени, за счет чего они могли набрать больше просмотров. Найдем самую позднюю дату и вычислим разницу в секундах для каждого объявления.

In [ ]:
max_date = X.start_time.max()
time = X.start_time.apply(lambda x: (max_date - x).total_seconds())

Добавим сгенерированные признаки, обучим модель и проверим качество прогноза:

In [ ]:
X_full = hstack([X_full,
                 time.values.reshape(-1,1),
                 title_len.values.reshape(-1,1)
                 ])

In [ ]:
X_train, X_holdout, y_train,  y_holdout = train_test_split(X_full.tocsr(), y, test_size=0.3, random_state=17)

In [ ]:
xgboost.fit(X_full, y_train)
rf.fit(X_train, y_train);

In [ ]:
xgboost.fit(X_train, y_train)

In [ ]:
predict = xgboost.predict(X_holdout)
rmse(y_holdout, predict)

In [ ]:
predict = rf.predict(X_holdout)
rmse(y_holdout, predict)

Прогноз немного улучшился

# 10. Построение кривых валидации и обучения

Построим кривые валидации и обучения для градиентного бустинга: 

In [ ]:
plot_learning_curve(xgboost, X_train_full, y_train, figsize=(10, 7), cv=10, scoring=rmse_scorer);

И для смешанного леса:

In [ ]:
plot_learning_curve(rf, X_train_full, y_train, figsize=(10, 7), cv=10, scoring=rmse_scorer);

Как видим увеличение данных с некоторого порога не приводит к значительному улучшению качества прогноза. Улучшить прогноз можно за счет новых признаков.

# 11. Прогноз для отложенной выборке

Протестируем лучшую модель на отложенной выборке

In [ ]:
predict = rf.predict(X_holdout)

In [ ]:
rmse(y_holdout, predict)

Результат на отложенной выборке сопоставим с результатами на кросс-валидации.

# 12. Выводы

Мы уже говорили о том, что знание количества просмотров может сделать сервис лучше за счет:
- оптимизации сайта 
- размещении рекламы на страницах наиболее просматриваемых объявлений
- устранения причин небольшоего количества просмотров "проблемных" объявлений
- информации о рынке (куда он движется и в какую сторону развиваться сервису?)

Построенную модель могут применять сотрудники Avito для предсказания количества просмотров объявления, и, следовательно, для решения задач, перечисленных выше.

Совершенству нет пределов! Дальнейшие пути улучшения решения: 
- собрать дополнительные признаки об объявлениях
- сгенерировать новые признаки на основе имеющихся
- объединить несколько прогнозных моделей